In [49]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!unzip -o "/content/drive/My Drive/HACKATHON/annotations_osf.zip" -d /content/


Archive:  /content/drive/My Drive/HACKATHON/annotations_osf.zip
  inflating: /content/annotations_osf/classes.txt  
  inflating: /content/annotations_osf/enhanced_01.jpg  
  inflating: /content/annotations_osf/enhanced_01.txt  
  inflating: /content/annotations_osf/enhanced_02.jpg  
  inflating: /content/annotations_osf/enhanced_02.txt  
  inflating: /content/annotations_osf/enhanced_03.jpg  
  inflating: /content/annotations_osf/enhanced_04.jpg  
  inflating: /content/annotations_osf/enhanced_05.jpg  
  inflating: /content/annotations_osf/enhanced_05.txt  
  inflating: /content/annotations_osf/enhanced_06.jpg  
  inflating: /content/annotations_osf/enhanced_07.jpg  
  inflating: /content/annotations_osf/enhanced_08.jpg  
  inflating: /content/annotations_osf/enhanced_09.jpg  
  inflating: /content/annotations_osf/enhanced_10.jpg  
  inflating: /content/annotations_osf/enhanced_100.jpg  
  inflating: /content/annotations_osf/enhanced_100.txt  
  inflating: /content/annotations_osf/enha

In [51]:
import os
import shutil

src = "/content/annotations_osf"
filtered = "/content/filtered_osf"
os.makedirs(filtered, exist_ok=True)

for fname in os.listdir(src):
    if fname.endswith('.jpg'):
        txt_name = fname.replace('.jpg', '.txt')
        if txt_name in os.listdir(src):  # if label exists
            shutil.copy(os.path.join(src, fname), filtered)
            shutil.copy(os.path.join(src, txt_name), filtered)

print("Filtered OSF images and annotations ready.")


Filtered OSF images and annotations ready.


In [52]:
import random

os.makedirs("/content/dataset_osf/train/images", exist_ok=True)
os.makedirs("/content/dataset_osf/train/labels", exist_ok=True)
os.makedirs("/content/dataset_osf/valid/images", exist_ok=True)
os.makedirs("/content/dataset_osf/valid/labels", exist_ok=True)

images = [f for f in os.listdir(filtered) if f.endswith('.jpg')]
random.shuffle(images)

split_idx = int(0.8 * len(images))
train_images = images[:split_idx]
val_images = images[split_idx:]

for img_list, subset in [(train_images, "train"), (val_images, "valid")]:
    for img in img_list:
        label = img.replace(".jpg", ".txt")
        shutil.copy(os.path.join(filtered, img), f"/content/dataset_osf/{subset}/images/")
        shutil.copy(os.path.join(filtered, label), f"/content/dataset_osf/{subset}/labels/")

print(f"Train images: {len(train_images)}, Valid images: {len(val_images)}")


Train images: 57, Valid images: 15


In [53]:
!zip -r /content/dataset_osf.zip /content/dataset_osf


updating: content/dataset_osf/ (stored 0%)
updating: content/dataset_osf/train/ (stored 0%)
updating: content/dataset_osf/train/labels/ (stored 0%)
updating: content/dataset_osf/train/labels/enhanced_23.txt (deflated 29%)
updating: content/dataset_osf/train/labels/enhanced_40.txt (deflated 48%)
updating: content/dataset_osf/train/labels/enhanced_63.txt (deflated 15%)
updating: content/dataset_osf/train/labels/enhanced_59.txt (deflated 27%)
updating: content/dataset_osf/train/labels/enhanced_92.txt (deflated 10%)
updating: content/dataset_osf/train/labels/enhanced_97.txt (deflated 10%)
updating: content/dataset_osf/train/labels/enhanced_88.txt (deflated 10%)
updating: content/dataset_osf/train/labels/enhanced_69.txt (deflated 10%)
updating: content/dataset_osf/train/labels/enhanced_41.txt (deflated 46%)
updating: content/dataset_osf/train/labels/enhanced_22.txt (deflated 29%)
updating: content/dataset_osf/train/labels/enhanced_47.txt (deflated 27%)
updating: content/dataset_osf/train/la

In [54]:
import os

# Clone YOLOv5 repository
if not os.path.exists("/content/yolov5"):
    !git clone https://github.com/ultralytics/yolov5.git /content/yolov5


In [55]:
!pip install -r /content/yolov5/requirements.txt


In [56]:
%cd /content/yolov5

/content/yolov5


In [59]:
yaml_content = """
train: /content/dataset_osf/train/images
val: /content/dataset_osf/valid/images

nc: 18
names:
  - dog
  - person
  - cat
  - tv
  - car
  - meatballs
  - marinara sauce
  - tomato soup
  - chicken noodle soup
  - french onion soup
  - chicken breast
  - ribs
  - pulled pork
  - hamburger
  - cavity
  - helmet
  - no helmet
  - he
"""

with open('/content/yolov5/osf_helmet.yaml', 'w') as f:
    f.write(yaml_content)

print("YAML file created: osf_helmet.yaml")


YAML file created: osf_helmet.yaml


In [62]:
!python train.py \
  --img 416 \
  --batch 8 \
  --epochs 30 \
  --data /content/yolov5/osf_helmet.yaml \
  --weights yolov5s.pt \
  --name yolo_osf_experiment


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-27 07:56:19.010177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753602979.062910   18943 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753602979.078509   18943 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: shailvisuman (shailvisuman-iit-madras-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/osf_helmet.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, img